- GAN主要包括了两个部分，即生成器generator与判别器discriminator。生成器主要用来学习真实图像分布从而让自身生成的图像更加真实，以骗过判别器。判别器则需要对接收的图片进行真假判别。在整个过程中，生成器努力地让生成的图像更加真实，而判别器则努力地去识别出图像的真假，这个过程相当于一个二人博弈，随着时间的推移，生成器和判别器在不断地进行对抗，最终两个网络达到了一个动态均衡：生成器生成的图像接近于真实图像分布，而判别器识别不出真假图像，对于给定图像的预测为真的概率基本接近0.5（相当于随机猜测类别）。

<img src="https://cdn.nlark.com/yuque/0/2021/jpeg/1508544/1614608345255-91f0b01a-0c17-4fe9-8aae-45e5ff927cdf.jpeg"/>

- 我们现在拥有大量的手写数字的数据集，希望通过GAN生成一些能够以假乱真的手写字图片。主要有如下两部分组成：  
（1）定义一个模型作为生成器（Generator），能够输入一个向量，输出手写数字大小的像素图像。  
（2）定义一个分类器作为判别器（Discriminator）用来图片是真的还是假的（或者说是来自数据集中的还是生成器中生成的），输入为手写图片，输出为判别图片的标签。

带你走进生成对抗网络的世界

In [1]:
import torch
from torch import nn
from torch.autograd.variable import Variable

import torchvision
import torchvision.transforms as transforms

为MNIST训练数据准备DataLoader，为这10个数字建立标签。

In [2]:
# Preprocess
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]
)

代码成功从天池实验室点击编辑按钮加载到DSW，加载好的代码会⾃动打开，默认在<b>download</b>
⽬录下<br>
1、点击左侧的【天池】按钮<br>
2、会出现【保存到天池】按钮和【添加数据源】模块，搜索Pytorch_MNIST，点击数据集中的下载按钮即可<br>
###### （具体如下图所示）
<center><img
src="https://img.alicdn.com/imgextra/i4/O1CN01zsetgx1zaOBQbSDLs_!!6000000006730-2-
tps-616-589.png" width=60%></center>
核⼼问题2
数据集下载成功后，⻚⾯右上⻆会提示数据集下载成功，也会说名数据集存储位置，默认在
<b>download</b>⽬录下，如下图所示。
<center>
<img
src="https://img.alicdn.com/imgextra/i3/O1CN01uJzjgf1MLwg6jK7za_!!6000000001419-2-tps-
1409-377.png" width=60%>
<img
src="https://img.alicdn.com/imgextra/i1/O1CN01XQmAP027k1R811xls_!!6000000007834-2-
tps-857-465.png" width=60%>
</center>

In [4]:
# Training data
train_set = torchvision.datasets.MNIST(
    root='.', train=True, download=True, transform=transform
)
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=32, shuffle=True
)

In [5]:
# Labels
classes = [str(i) for i in range(0, 10)]

判别器部分。  判别器网络是对图像真实与否进行分类。  
输入：28×28像素 -> 一个长度为784的向量    
输出：一个单独的值<表示图像是否是实际的MNIST数字>

In [6]:
# Our Discriminator classes
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.model(x.view(x.size(0), 784))
        out = out.view(out.size(0), -1)
        return out.cuda()

discriminator = Discriminator()

生成器部分。  生成器网络负责创建实际的图像。  
输入：一个长度为100的向量<纯随机噪声>  
输出：一个长度为784的向量 -> 28×28像素

In [7]:
# Our Generator class
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 784),
            nn.Tanh()
        )

    def forward(self, x):
        x = x.view(x.size(0), 100)
        out = self.model(x).cuda()
        return out

generator = Generator()

把模型移动到GPU上。

In [8]:
# If we have a GPU with CUDA, use it
if torch.cuda.is_available():
    print("Using CUDA")
    discriminator.cuda()
    generator.cuda()

# Setup loss function and optimizers
lr = 0.01
num_epochs = 40
num_batches = len(train_loader)

criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)

Using CUDA


开始训练循环。  
训练GAN的关键是我们需要在一个循环中更新生成器和判别器。

In [9]:
# Convenience function for training our Discriminator
def train_discriminator(discriminator, real_images, real_labels, fake_images, fake_labels):
    discriminator.zero_grad()

    # Get the predictions, loss, and score of the real images
    predictions = discriminator(real_images)
    real_loss = criterion(predictions, real_labels)
    real_score = predictions

    # Get the predictions, loss, and score of the fake images
    predictions = discriminator(fake_images)
    fake_loss = criterion(predictions, fake_labels)
    fake_score = predictions

    # Calculate the total loss, update the weights, and update the optimizer
    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()
    return d_loss, real_score, fake_score

In [10]:
# Convenience function for training our Generator
def train_generator(generator, discriminator_outputs, real_labels):
    generator.zero_grad()

    # Calculate the total loss, update the weights, and update the optimizer
    g_loss = criterion(discriminator_outputs, real_labels)
    g_loss.backward()
    g_optimizer.step()
    return g_loss

In [11]:
for epoch in range(num_epochs):
    for n, (images, _) in enumerate(train_loader):
        # (1) Prepare the real data for the Discriminator
        real_images = Variable(images).cuda()
        real_labels = Variable(torch.ones(images.size(0))).reshape([32, 1]).cuda()

        # (2) Prepare the random noise data for the Generator
        noise = Variable(torch.randn(images.size(0), 100)).cuda()

        # (3) Prepare the fake data for the Discriminator
        fake_images = generator(noise)
        fake_labels = Variable(torch.zeros(images.size(0))).reshape([32, 1]).cuda()

        # (4) Train the discriminator on real and fake data
        d_loss, real_score, fake_score = train_discriminator(
            discriminator,
            real_images, real_labels,
            fake_images, fake_labels
        )

        # (5a) Generate some new fake images from the Generator.
        # (5b) Get the label predictions of the Discriminator on that fake data.
        noise = Variable(torch.randn(images.size(0), 100)).cuda()
        fake_images = generator(noise)

        outputs = discriminator(fake_images)

        # (6) Train the generator
        g_loss = train_generator(generator, outputs, real_labels)

# 作业

CGAN的诞生，对GAN的应用起到了极大的推广，使得GAN的变种多了许多，但是其基础还是CGAN。  
实现GAN、CGAN在MNIST上的实验，并比较它们之间的优缺点。  

<center><img src="https://cdn.nlark.com/yuque/0/2021/png/1508544/1613296043422-e95ecabb-39c4-4da0-a866-26374241f29f.png" style="zoom:80%" /><img src="https://cdn.nlark.com/yuque/0/2021/png/1508544/1613296047452-e533f474-ebb5-4d66-9e9e-de9e36b4ba83.png" style="zoom:80%" /><img src="https://cdn.nlark.com/yuque/0/2021/png/1508544/1613296051715-35fdd763-6e91-4e48-93ce-e8815a644df4.png" style="zoom:80%" /></center>

<center>GAN在MNIST上实验结果</center>

<center><img src="https://cdn.nlark.com/yuque/0/2021/png/1508544/1613296082069-7d80d0d6-2876-49a5-9ac3-3fcfa705fbd2.png" style="zoom:80%" /><img src="https://cdn.nlark.com/yuque/0/2021/png/1508544/1613296085509-64f49d13-c243-46a5-8c45-d58321a69c2b.png" style="zoom:80%" /><img src="https://cdn.nlark.com/yuque/0/2021/png/1508544/1613296088257-0f6f6a97-2a76-4f8e-bbaf-a118f6bce2ca.png" style="zoom:80%" /></center>

<center>CGAN在MNIST上实验结果</center>